# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [384]:
from statistics import linear_regression, LinearRegression

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import sklearn
import re

from scipy.constants import horsepower
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [385]:
# Model imports
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [386]:
# Filepath
file_path = "data/test.csv"
test_data = pd.read_csv(file_path)
#added for a quick way to see some of the data
test_data.head(5)


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [387]:
# Filepath for train data
file_path = "data/train.csv"
train_data = pd.read_csv(file_path)
#added for a quick way to see some of the data
train_data.head(5)


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [388]:
# note to self may need to split the train data in to some thing like 70-30% for training and testing. Since the test data does not contain prices
# Split the data in to training and testing data
tmp1 = train_data.drop(columns=["price"])
tmp2 = train_data["price"]
X_train, X_test, Y_train, Y_test = train_test_split(tmp1, tmp2, test_size=0.2, random_state=42) # split the data in to 80% training and 20% testing
# X_train, X_test, Y_train, Y_test = train_test_split(train_data_file_path, test_size=0.3, random_state=42) # split the data in to 70% training and 30% testing
print("train data (rows, columns):", X_train.shape)

train data (rows, columns): (150826, 12)


In [389]:
# Variables from the data
# aka the different columns in the data. So i can use them in the model, the preprocessing, in the evaluation and so i can see what is the most important for the price of the cars
# The variables are the columns in the data

#target variable. Aka the price of the car
target = "price" # the price of the car. What we are trying to predict
id_col = "id" # the id of the car
brand_col = "brand" # the brand of the car
model_year_col = "model_year" # the year the car was made
mileage_col = "mileage" # the mileage of the car
engine_col = "engine" # the engine of the car. Could be useful for the price and to split the data in two categories in the future
transmission_col = "transmission" # transmission of the car. The main diffrence is automatic and manual. That could be useful for the price and to split the data in to two main categories in the future. Cat wolud be manual or automatic
ext_color_col = 'ext_col' # Color of the car outside.
int_color_col = "int_col" # Interior color of the car. <-- note is more important in some climates than others (aka in hot climates with lots of sun the dark interior colors can get very hot)
accident_col = "accident" # If the car has been in an accident. <--- Note to self this could be a good variable to use in the model. Since cars that have been in accidents are usually cheaper and are more risky to buy
clean_title_col = "clean_title" # If the title of the car is clean. <--- Note to self this could be a good variable to use in the model. Since cars that have clean titles are usually more expensive and are less risky to buy
model_col = "model" # the model of the car. Could be useful for the price and to split the data in to two categories in the future
fuel_col = "fuel_type" # the fuel type of the car. Could be useful for the price and to split the data in to two categories in the future


print("drop, price, colors (for now interior and exterior) and clean title column") #check clean title column if it is useful later as well as the colors
print(train_data_file_path.shape)
# x and y
X = train_data_file_path.drop(columns=[target, ext_color_col, int_color_col, clean_title_col])
y = train_data_file_path[target]

print("test data (rows, columns):", X.shape)

drop, price, colors (for now interior and exterior) and clean title column
(188533, 13)
test data (rows, columns): (188533, 9)


In [390]:
# Methods for the model
# The methods are the different functions that can be used in the model
# So the methods I need to make is to identify the diffrent information in the columns and to preprocess the data.

# Method to extract the year from the model year column
def extract_year(df):
    df[model_year_col] = df[model_year_col].str.extract(r'(\d{4})').astype(int)
    return df

# Method to extract the engine from the engine column
def extract_engine(df):
    df[engine_col] = df[engine_col].str.extract(r'(\d+)').astype(float)
    return df
# Method to exstract horsepower from the engine column
# def extract_hp(df):
#     if pd.isnull(df):
#         return np.nan
#     # horsepower
#     hp = re.search(r'(\d+)(?=hp)', df) # search for the horsepower. May need to change the regex
#     hp = float(hp.group(0)) \
#         if hp \
#         else np.nan
#     return hp

def extract_hp(df):
    if pd.isnull(df):
        return np.nan

    # Ensure the input is a string
    df = str(df)

    # Improving the regex to capture different formats and case variations for 'hp' and 'HP'
    hp = re.search(r'(\d+)\s*[hH][pP]', df)

    # Extract and return the horsepower as a float
    hp = float(hp.group(1)) if hp else np.nan

    return hp

# Method to get out accident information for the accident column
def extract_accident(df): #, accident_col
    # if accident_col not in df.columns:
    #     raise ValueError(f"Column '{accident_col}' not found in DataFrame")
    #
    # df[accident_col] = df[accident_col].str.lower().str.contains("yes").astype(int) # if the car has been in an accident
    # return df # return the data frame
    if pd.isnull(df):
        return 0
    return 0 if df.strip().lower() == "none reported" else 1

# Method to get out the transmission information for the transmission column
def extract_transmission(df):
    # if the car has automatic transmission or manual transmission or non reported
    if df[transmission_col].str.contains("automatic").any():
        df[transmission_col] = 1
    else:
        df[transmission_col] = 0
    return df


In [391]:
# Apply the methods to the data

# Apply horsepower method
X[engine_col] = X[engine_col].apply(extract_hp)
X[engine_col] = X[engine_col].fillna(X[engine_col].mean())
print("test size after engine method:", X.shape)

# Apply the extract_accident method
X[accident_col] = X[accident_col].apply(extract_accident)
# X = extract_accident(X, accident_col)
print("test size after accident method:", X.shape)
# print(X)

test size after engine method: (188533, 9)
test size after accident method: (188533, 9)


In [392]:
# Thresholds
# Threshold for brand category
brand_threshold = 100
# Count the number of cars for each brand
brand_cou = X[brand_col].value_counts()

# Brand
brand_cat = brand_cou[brand_cou > brand_threshold].index
# Brand category. Group the less frequent brands into "Other"
X[brand_col] = X[brand_col].apply(lambda x: x if x in brand_cat else "Other")
X =  pd.get_dummies(X, columns=[brand_col])

# Threshold for color category. Exterior color and interior color
color_threshold_ext = 100
color_threshold_int = 100

# # Exterior color
# ext_color_cat = X[ext_color_col].value_counts()
# # Interior color
# int_color_cat = X[int_color_col].value_counts()
#
# # Identify if the color is the same
# ext_color_common = ext_color_cat[ext_color_cat > color_threshold_ext].index
# int_color_common = int_color_cat[int_color_cat > color_threshold_int].index
#
# # Exterior color category. Group the less frequent colors into "Other"
# X[ext_color_col] = X[ext_color_col].where(X[ext_color_col].isin(ext_color_common), "Other")
# # X[ext_color_col] = X[ext_color_col].apply(lambda x: x if x in ext_color_cat else "Other")
# X =  pd.get_dummies(X, columns=[ext_color_col])
# # Interior color category. Group the less frequent colors into "Other"
# X[int_color_col] = X[int_color_col].where(X[int_color_col].isin(int_color_common), "Other")
# # X[int_color_col] = X[int_color_col].apply(lambda x: x if x in int_color_cat else "Other")
# X =  pd.get_dummies(X, columns=[int_color_col])
#
# print("common exterior colors:", ext_color_common)
# print("common interior colors:", int_color_common)
# print(len(ext_color_common))
# print(len(int_color_common))

# Threshold for transmission category. Automatic and manual. Vet ikke helt hvike type kategorier er det som blir her
transmission_threshold = 100

# Transmission
transmission_cat = X[transmission_col].value_counts()

# Identify if the transmission is the same
transmission_common = transmission_cat[transmission_cat > transmission_threshold].index

# Transmission category. Group the less frequent transmissions into "Other"
X[transmission_col] = X[transmission_col].where(X[transmission_col].isin(transmission_common), "Other")

X =  pd.get_dummies(X, columns=[transmission_col])

# Threshold for fuel type category
fuel_threshold = 100
# Count the number of cars for each fuel type
fuel_couts = X[fuel_col].value_counts()
fuel_common = fuel_couts[fuel_couts > fuel_threshold].index
print("common fuel types:", fuel_common)
X[fuel_col] = X[fuel_col].where(X[fuel_col].isin(fuel_common), "Other")
X =  pd.get_dummies(X, columns=[fuel_col])

# Threshold model
model_threshold = 100
# Count the number of cars for each model
model_cat = X[model_col].value_counts()
model_common = model_cat[model_cat > model_threshold].index
X[model_col] = X[model_col].where(X[model_col].isin(model_common), "Other")
X =  pd.get_dummies(X, columns=[model_col])

print("common models:", model_common)
print("test size after thresholding:", X.shape)




common fuel types: Index(['Gasoline', 'Hybrid', 'E85 Flex Fuel', 'Diesel', '–', 'Plug-In Hybrid'], dtype='object', name='fuel_type')
common models: Index(['F-150 XLT', 'M3 Base', 'Camaro 2SS', 'M4 Base', 'Mustang GT Premium',
       'F-150 Lariat', 'E-Class E 350 4MATIC', '1500 Laramie', '911 Carrera S',
       'Corvette Base',
       ...
       'Sorento SXL', 'Suburban RST', 'X5 PHEV xDrive45e', 'Nautilus Reserve',
       'G70 2.0T', 'Santa Fe SEL', '2500 SLT', 'Bronco Sport Outer Banks',
       'Stinger GT2', 'Bronco Outer Banks'],
      dtype='object', name='model', length=475)
test size after thresholding: (188533, 565)


In [393]:
# Fit and transform the training data.



In [394]:
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

# Model
gradient_boosting = GradientBoostingClassifier()
# linear_reg = LinearRegression()
random_forest_reg = RandomForestRegressor()

# Preprocessor
preprocessor = ColumnTransformer( transformers=[('numerical', StandardScaler(),X.columns)])

xgb = XGBClassifier( # Heads up only works in PyCharmPro ide not the notebook web version
    scale_pos_weight=len(Y_train[Y_train == 0]),
    n_estimators=10000,
    random_state=42,
    subsample=0.9,
    min_child_weight=10,
    max_depth=6,
    learning_rate=0.01,
    gamma=0.5,
    colsample_bytree=0.9,
)

model = Pipeline([('preprocessor', preprocessor), ('classifier', xgb)])


In [395]:

# Train with fit
# gradient_boosting.fit(X_train, Y_train)
# linear_reg.fit(X_train, Y_train)
random_forest_reg.fit(X_train, Y_train)

model.fit(X_train, Y_train)


ValueError: could not convert string to float: 'Mercedes-Benz'

In [ ]:
# Predict
gradient_boosting_pred = gradient_boosting.predict(X_test)
# linear_reg_pred = linear_reg.predict(X_test)
random_forest_reg_pred = random_forest_reg.predict(X_test)

model_pred = model.predict(X_test)
data_test_pred = model.predict(test_data)


In [ ]:
# Evaluate
# Evaluate the model

# mean = mean_squared_error(Y_test, linear_reg_pred)

mean = mean_squared_error(Y_test, random_forest_reg_pred)
root_m = root_mean_squared_error(mean)

print(f"accuracy: {accuracy_score(Y_test, gradient_boosting_pred)}")
print("Mean Squared Error:", mean)
print("Root Mean Squared Error:", root_m)
